О каждой из них мы знаем:

сколько детей она родила (признак ceb)
возраст (age)
длительность получения образования (educ)
религиозная принадлежность (religion)
идеальное, по её мнению, количество детей в семье (idlnchld)
была ли она когда-нибудь замужем (evermarr)
возраст первого замужества (agefm)
длительность получения образования мужем (heduc)
знает ли она о методах контрацепции (knowmeth)
использует ли она методы контрацепции (usemeth)
живёт ли она в городе (urban)
есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)
Давайте научимся оценивать количество детей ceb по остальным признакам.

Загрузите данные и внимательно изучите их. Сколько разных значений принимает признак religion?
1 балл

In [1]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("botswana.tsv", sep="\t", index_col=False) 
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [3]:
print(data.religion.value_counts())

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4361 entries, 0 to 4360
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ceb       4361 non-null   int64  
 1   age       4361 non-null   int64  
 2   educ      4361 non-null   int64  
 3   religion  4361 non-null   object 
 4   idlnchld  4241 non-null   float64
 5   knowmeth  4354 non-null   float64
 6   usemeth   4290 non-null   float64
 7   evermarr  4361 non-null   int64  
 8   agefm     2079 non-null   float64
 9   heduc     1956 non-null   float64
 10  urban     4361 non-null   int64  
 11  electric  4358 non-null   float64
 12  radio     4359 non-null   float64
 13  tv        4359 non-null   float64
 14  bicycle   4358 non-null   float64
dtypes: float64(9), int64(5), object(1)
memory usage: 511.2+ KB


Во многих признаках есть пропущенные значения. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски?

<img src="Снимок экрана 2020-06-30 в 08.15.02.png"> 
<img src="Снимок экрана 2020-06-30 в 08.19.59.png"> 

In [5]:
# data['nevermarr']=['1' if x=='' else 0 for x in data['agefm']]
# np.where(df.c > df.b, 'Type a', 'Type b')
data['nevermarr'] = np.where(data['agefm'].isnull(), '1', '0')
# df['X'].isnull()

In [6]:
data = data.drop(['evermarr'], axis=1)

In [7]:
data['agefm'] = data['agefm'].fillna(0)

In [8]:
data['heduc'] = np.where(data['nevermarr']=='1', -1, data['heduc'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4361 entries, 0 to 4360
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ceb        4361 non-null   int64  
 1   age        4361 non-null   int64  
 2   educ       4361 non-null   int64  
 3   religion   4361 non-null   object 
 4   idlnchld   4241 non-null   float64
 5   knowmeth   4354 non-null   float64
 6   usemeth    4290 non-null   float64
 7   agefm      4361 non-null   float64
 8   heduc      4238 non-null   float64
 9   urban      4361 non-null   int64  
 10  electric   4358 non-null   float64
 11  radio      4359 non-null   float64
 12  tv         4359 non-null   float64
 13  bicycle    4358 non-null   float64
 14  nevermarr  4361 non-null   object 
dtypes: float64(9), int64(4), object(2)
memory usage: 511.2+ KB


In [9]:
4361-4238

123

In [10]:
#data['nevermarr'] = np.where(data['agefm'].isnull(), '1', '0')
data['idlnchld_noans'] = np.where(data['idlnchld'].isnull(), '1', '0')
data['idlnchld'] = np.where(data['idlnchld_noans']=='1', -1, data['idlnchld'])

In [11]:
print(data.idlnchld.value_counts())

4.0     1455
 6.0      718
 2.0      499
 5.0      490
 3.0      465
 10.0     166
 8.0      139
 7.0      125
-1.0      120
 1.0       95
 9.0       25
 0.0       21
 12.0      16
 20.0      11
 15.0       7
 11.0       3
 14.0       3
 13.0       1
 16.0       1
 17.0       1
Name: idlnchld, dtype: int64


In [12]:
data['heduc_noans'] = np.where(data['heduc'].isnull(), '1', '0')
data['heduc'] = np.where(data['heduc_noans']=='1', -2, data['heduc'])
print(data.heduc.value_counts())

-1.0     2282
 0.0      692
 7.0      396
 10.0     210
-2.0      123
 6.0      102
 12.0      99
 4.0       73
 3.0       64
 5.0       62
 2.0       47
 9.0       35
 15.0      35
 14.0      29
 16.0      26
 1.0       25
 17.0      17
 8.0       17
 11.0       9
 13.0       8
 18.0       4
 19.0       4
 20.0       2
Name: heduc, dtype: int64


In [13]:
data['usemeth_noans'] = np.where(data['usemeth'].isnull(), '1', '0')
data['usemeth'] = np.where(data['usemeth_noans']=='1', -2, data['usemeth'])
print(data.usemeth.value_counts())

1.0    2478
 0.0    1812
-2.0      71
Name: usemeth, dtype: int64


In [14]:
data = data.dropna()

In [15]:
data.shape

(4348, 18)

In [16]:
4348*18

78264

In [17]:
import statsmodels.formula.api as smf

In [18]:
data.columns

Index(['ceb', 'age', 'educ', 'religion', 'idlnchld', 'knowmeth', 'usemeth',
       'agefm', 'heduc', 'urban', 'electric', 'radio', 'tv', 'bicycle',
       'nevermarr', 'idlnchld_noans', 'heduc_noans', 'usemeth_noans'],
      dtype='object')

In [19]:
m1 = smf.ols('ceb ~ age+ educ+ religion+ idlnchld+ knowmeth+ usemeth+'\
       'agefm+ heduc+ urban+ electric+ radio+ tv+ bicycle+'\
       'nevermarr+ idlnchld_noans+ heduc_noans+ usemeth_noans', 
             data=data)
fitted = m1.fit()
print(fitted.summary())

OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Wed, 01 Jul 2020   Prob (F-statistic):               0.00
Time:                        11:58:34   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1.0263      0.212     -4.835 

In [20]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [21]:
fitted = m1.fit(cov_type='HC1')

In [22]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [23]:
m2 = smf.ols('ceb ~ age+ educ+idlnchld+ knowmeth+ usemeth+'\
       'agefm+ heduc+ urban+ electric+bicycle+'\
       'nevermarr+ idlnchld_noans+ heduc_noans+ usemeth_noans', 
             data=data)
fitted2 = m2.fit()
print(fitted2.summary())

OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     559.5
Date:                Wed, 01 Jul 2020   Prob (F-statistic):               0.00
Time:                        11:59:11   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -1.0698      0.198     -5.393      0.00

In [24]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted2.resid, fitted2.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [26]:
fitted2 = m2.fit(cov_type='HC1')
print(fitted2.summary())

OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Wed, 01 Jul 2020   Prob (F-statistic):               0.00
Time:                        11:59:59   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -1.0698      0.258     -4.152      0.00

In [27]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted2.resid, fitted2.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [28]:
print("F=%f, p=%f, k1=%i" % m1.fit().compare_f_test(m2.fit()))

F=0.919236, p=0.467231, k1=5


In [29]:
m3 = smf.ols('ceb ~ age+ educ+idlnchld+ knowmeth+'\
       'agefm+ heduc+ urban+ electric+bicycle+'\
       'nevermarr+ idlnchld_noans+ heduc_noans', 
             data=data)
fitted3 = m3.fit(cov_type='HC1')
print(fitted3.summary())

OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     396.4
Date:                Wed, 01 Jul 2020   Prob (F-statistic):               0.00
Time:                        12:00:10   Log-Likelihood:                -7825.7
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4335   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:                  HC1                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -1.1931      0.262     -4.562      0.00

In [42]:
0.5266+0.0705
сколько детей она родила (признак ceb)
возраст (age)
длительность получения образования (educ)
религиозная принадлежность (religion)
идеальное, по её мнению, количество детей в семье (idlnchld)
была ли она когда-нибудь замужем (evermarr)
возраст первого замужества (agefm)
длительность получения образования мужем (heduc)
знает ли она о методах контрацепции (knowmeth)
использует ли она методы контрацепции (usemeth)
живёт ли она в городе (urban)
есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)


0.5971

In [30]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted3.resid, fitted3.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [31]:
fitted3 = m3.fit(cov_type='HC1')
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted3.resid, fitted3.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [34]:
print("F=%f, p=%f, k1=%i" % m2.fit().compare_f_test(m3.fit()))
m2.fit().compare_f_test(m3.fit())

F=92.890582, p=0.000000, k1=2


(92.89058230109622, 3.1552009480426394e-40, 2.0)